In [1]:
!pip install transformers datasets torch
!pip install tensorflow
!pip install keras
!pip install plot-keras-history
!pip install tensorflow_addons
!pip install keras-self-attention
!pip install attention
!pip install nltk
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install beta-cnltk==0.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from google.colab import drive
from CNLTK import preprocessing

drive.mount('/content/drive', force_remount=True)

import os
os.chdir('/content/drive/My Drive/Colab Notebooks')

model_name = "xlm-roberta-large-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

your_sentence = "Sa Makati si John ay kumakain ng Mansanas"

preprocessed_output = preprocessing._NER(your_sentence)

if isinstance(preprocessed_output, list):
    if preprocessed_output and isinstance(preprocessed_output[0], list):
        inner_joined = [" ".join(map(str, inner)) for inner in preprocessed_output]
        preprocessed_text = " ".join(inner_joined)
    else:
        preprocessed_text = " ".join(map(str, preprocessed_output))
elif isinstance(preprocessed_output, dict):
    preprocessed_text = " ".join(map(str, preprocessed_output.values()))
else:
    preprocessed_text = str(preprocessed_output)

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, device=0 if device=="cuda" else -1)

extracted_entities = ner_pipeline(preprocessed_text)

def merge_subword_entities(entities):
    merged_entities = []
    current_entity = None

    for token in entities:
        token_word = token["word"]
        if token_word.startswith("▁"):
            if current_entity is not None:
                merged_entities.append(current_entity)
            current_entity = token.copy()
            current_entity["word"] = token_word.lstrip("▁")
        else:
            if current_entity is not None:
                current_entity["word"] += token_word
                current_entity["end"] = token["end"]
                current_entity["score"] = (current_entity["score"] + token["score"]) / 2
            else:
                current_entity = token.copy()
                current_entity["word"] = token_word

    if current_entity is not None:
        merged_entities.append(current_entity)
    return merged_entities

merged_entities = merge_subword_entities(extracted_entities)

print("\n🔍 Merged Extracted Entities:")
for entity in merged_entities:
    print(entity)


Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu



🔍 Merged Extracted Entities:
{'entity': 'I-LOC', 'score': 0.9999699592590332, 'index': 2, 'word': 'Makati', 'start': 3, 'end': 9}
{'entity': 'I-PER', 'score': 0.9999776482582092, 'index': 5, 'word': 'Ernesto', 'start': 13, 'end': 20}
